# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- hoursPerWindow: 2


# Preproccess

## read data

In [3]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [4]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [5]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 917
ph 954
po2 917
albumin 406
hba1c 326
lymphocyte 446
height 415
urine-ketone 294
crp 19


In [6]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1209

## split patients

In [7]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [8]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,race,rr,saps2,sbp,scr,sofa,use_NaHCO3,uti,wbc,weight
0,19054290,20046699,30643955,False,NaN,65,NaN,NaN,NaN,NaN,...,WHITE,NaN,34,NaN,NaN,3,0.0,0,NaN,74.50
0,14866589,20066152,33060916,False,NaN,37,NaN,NaN,NaN,NaN,...,WHITE,NaN,22,NaN,NaN,2,0.0,0,NaN,NaN
0,14849360,20152551,32817197,False,NaN,59,NaN,NaN,NaN,NaN,...,UNABLE TO OBTAIN,NaN,31,NaN,NaN,7,0.0,0,NaN,NaN
0,16171124,20167052,38239449,False,NaN,37,NaN,NaN,NaN,NaN,...,HISPANIC/LATINO - DOMINICAN,NaN,15,NaN,NaN,2,0.0,0,NaN,NaN
0,16815664,20240670,37751533,False,NaN,42,NaN,NaN,NaN,NaN,...,BLACK/AFRICAN AMERICAN,NaN,27,NaN,NaN,2,NaN,0,NaN,43.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,15517352,29801212,32814591,False,NaN,20,NaN,NaN,NaN,NaN,...,WHITE,NaN,30,NaN,NaN,4,NaN,0,NaN,NaN
0,10383045,29899941,39755932,False,NaN,58,NaN,NaN,NaN,NaN,...,WHITE,NaN,23,NaN,NaN,1,0.0,0,NaN,59.40
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,WHITE,NaN,32,NaN,NaN,3,0.0,0,NaN,47.40
0,10912213,29911812,34040470,False,NaN,47,NaN,NaN,NaN,NaN,...,WHITE,NaN,23,NaN,NaN,2,0.0,0,NaN,89.05


In [9]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [10]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

967 242
967 242
967 242
967 242
968 241


In [11]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [12]:
import pickle


(TEMP_PATH / f"train-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

20724110

In [13]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


datavalset = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    datavalset.append((npTrainX, trainY, npValX, valY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [14]:
import pickle

(TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(datavalset)
)

20434200

# LSTM

In [15]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC", "accuracy", "precision", "recall"])
    return model

2024-06-07 12:11:27.411407: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-07 12:11:27.429298: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-07 12:11:27.699417: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

2024-06-07 12:11:27.904554: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


Epoch 1/50


2024-06-07 12:11:27.918956: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/tu/codepy/hust.year2023.PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 1/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - AUC: 0.5688 - accuracy: 0.5625 - loss: 0.6789 - precision: 0.4375 - recall: 0.5833

17/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.6348 - accuracy: 0.6002 - loss: 0.6683 - precision: 0.4851 - recall: 0.6054

...

22/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.9508 - accuracy: 0.8755 - loss: 0.2857 - precision: 0.8241 - recall: 0.8699 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.9500 - accuracy: 0.8752 - loss: 0.2870 - precision: 0.8229 - recall: 0.8708 - val_AUC: 0.7026 - val_accuracy: 0.6495 - val_loss: 0.8675 - val_precision: 0.5402 - val_recall: 0.6267


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.7949 - accuracy: 0.7188 - loss: 0.5501 - precision: 0.7059 - recall: 0.7500

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.7764 - accuracy: 0.7131 - loss: 0.5653 - precision: 0.6365 - recall: 0.6899


Loses: [0.5879510045051575, 0.5781559348106384, 0.4933958351612091, 0.6052817702293396, 0.5806713104248047] 0.5690911710262299 0.03901667908012893
AUCs: [0.7452560067176819, 0.7875760793685913, 0.84980309009552, 0.7561001777648926, 0.7611391544342041] 0.779974901676178 0.03758708098105382
Accuracies: [0.7314049601554871, 0.7148760557174683, 0.7851239442825317, 0.6983470916748047, 0.7095435857772827] 0.7278591275215149 0.030550979200059128
Precisions: [0.6595744490623474, 0.6065573692321777, 0.7047619223594666, 0.5950413346290588, 0.6213592290878296] 0.637458860874176 0.04008028988995268
Recals: [0.6526315808296204, 0.7789473533630371, 0.7789473533630371, 0.75, 0.6736842393875122] 0.7268421053886414 0.053477401748582364


## Static validate 

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )
    return model

In [19]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

if "datavalset" not in locals():
    datavalset = pickle.loads(
        (TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )

loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npValX, valY, npTestX, testY) in enumerate(datavalset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    valY = np.array(valY)
    testY = np.array(testY)

    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=(npValX, valY),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - AUC: 0.5148 - accuracy: 0.5938 - loss: 0.6500 - precision: 0.1667 - recall: 0.4000

18/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.5550 - accuracy: 0.5619 - loss: 0.6841 - precision: 0.4014 - recall: 0.4336

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - AUC: 0.5899 - accuracy: 0.5853 - loss: 0.6775 - precision: 0.4453 - recall: 0.4634 - val_AUC: 0.7457 - val_accuracy: 0.7047 - val_loss: 0.6075 - val_precision: 0.6234 - val_recall: 0.6316


Epoch 2/50


...

23/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.9270 - accuracy: 0.8682 - loss: 0.3282 - precision: 0.7781 - recall: 0.9184 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.9269 - accuracy: 0.8679 - loss: 0.3293 - precision: 0.7804 - recall: 0.9143 - val_AUC: 0.7750 - val_accuracy: 0.7047 - val_loss: 0.6790 - val_precision: 0.6022 - val_recall: 0.7368


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.7773 - accuracy: 0.8125 - loss: 0.5279 - precision: 0.7778 - recall: 0.8750

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.7714 - accuracy: 0.7473 - loss: 0.5602 - precision: 0.6602 - recall: 0.7764


Loses: [0.6015784740447998, 0.5485941171646118, 0.4690885841846466, 0.5539657473564148, 0.5780329704284668] 0.550251978635788 0.04474422827037178
AUCs: [0.7500537633895874, 0.7882563471794128, 0.8569638729095459, 0.7836400866508484, 0.758147120475769] 0.7874122381210327 0.037694759035235476
Accuracies: [0.6735537052154541, 0.7438016533851624, 0.7851239442825317, 0.7190082669258118, 0.7095435857772827] 0.7262062311172486 0.03710158684336455
Precisions: [0.5754716992378235, 0.6666666865348816, 0.7362637519836426, 0.6458333134651184, 0.6086956262588501] 0.6465862154960632 0.05466044987827452
Recals: [0.6421052813529968, 0.6947368383407593, 0.7052631378173828, 0.6458333134651184, 0.7368420958518982] 0.6849561333656311 0.03624080649741921
